# GoogLeNet的Pytorch实现

In [6]:
import time
import torch
from torch import nn, optim
import torchvision
from torchsummary import summary
import torch.nn.functional as F

## 定义模型

In [22]:
class Inception(nn.Module):
    def __init__(self, in_channels, conv1x1, conv3x3red, conv3x3, conv5x5red, conv5x5, pool_proj):
        super(Inception, self).__init__()
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=conv1x1, kernel_size=1),
            nn.ReLU()
        )
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=conv3x3red, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=conv3x3red, out_channels=conv3x3, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=conv5x5red, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=conv5x5red, out_channels=conv5x5, kernel_size=3, padding=1)
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=in_channels, out_channels=pool_proj, kernel_size=1),
            nn.ReLU()
        )

        '''# 线路1，单1 x 1卷积层
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1 x 1卷积层后接5 x 5卷积层
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3 x 3最大池化层后接1 x 1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size=1)'''

        
        
        
    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        output = torch.cat((branch1, branch2, branch3, branch4), dim=1)
        return output
        

class GoogLeNetModel(nn.Module):
    def __init__(self, classes):
        super(GoogLeNetModel, self).__init__()
        self.classes = classes
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)
        
        
        self.avgpool = nn.AvgPool2d(kernel_size=6)
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, self.classes)
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool1(x)
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool2(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = self.dropout(x)
        output = self.fc(x.view(x.shape[0], -1))
        return output

net = GoogLeNetModel(10)
summary(net, (3, 224, 224), device="cpu")
        
        
        
        
        
        
        

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
              ReLU-2         [-1, 64, 112, 112]               0
         MaxPool2d-3           [-1, 64, 55, 55]               0
            Conv2d-4           [-1, 64, 55, 55]           4,160
              ReLU-5           [-1, 64, 55, 55]               0
            Conv2d-6          [-1, 192, 55, 55]         110,784
              ReLU-7          [-1, 192, 55, 55]               0
         MaxPool2d-8          [-1, 192, 27, 27]               0
            Conv2d-9           [-1, 64, 27, 27]          12,352
             ReLU-10           [-1, 64, 27, 27]               0
           Conv2d-11           [-1, 96, 27, 27]          18,528
             ReLU-12           [-1, 96, 27, 27]               0
           Conv2d-13          [-1, 128, 27, 27]         110,720
             ReLU-14          [-1, 128,